Instacart Online Retail Analysis
-----
This is an Integrated CA2 for CCT College Higher Diploma in Data Analytics for Business.
Courses: Machine Learning for Business / Data Visualization Techniques.
Main goals for this Analysis are:

-Build basic recommender systems (content based filtering and collaborative filtering).

-Apply Market Basket Analysis (Apriori and FP Growth).

-Design an interactive dashboard aimed at adults (+65 Years)

Libraries importation

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import apriori, association_rules

from ipywidgets import interact, widgets

Display and plot pre settings

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.4f}'.format)

sns.set_theme(style="whitegrid")

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

Data Loading
---
In this section we are going to load four csv files and have a first look at their structure

In [4]:
orders = pd.read_csv('orders.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
products = pd.read_csv('products.csv')
departments = pd.read_csv('departments.csv')

orders.shape, order_products_prior.shape, products.shape , departments.shape

((3421083, 7), (32434489, 4), (49688, 4), (21, 2))

Now lets check the first rows of every dataset:

In [5]:
print("=== orders ===")
display(orders.head())

print("\n=== order_products_prior ===")
display(order_products_prior.head())

print("\n=== products ===")
display(products.head())

print("\n=== departments ===")
display(departments.head())


=== orders ===


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0000
2,473747,1,prior,3,3,12,21.0000
3,2254736,1,prior,4,4,7,29.0000
4,431534,1,prior,5,4,15,28.0000



=== order_products_prior ===


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



=== products ===


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13



=== departments ===


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


Now lets see the info for each of them to have an overview of what are we going to clean and merge later on

In [6]:
orders.info(), 
order_products_prior.info(),
products.info(),
departments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  -

Merging Tables
---
Now we are going to merge this tables in order to get one single dataset, first we are going to merge "orders" with "order_products_prior" by order_id column

In [7]:
orders_prior = orders.merge(order_products_prior, on="order_id", how="inner")

print("Merged (orders + prior) shape:", orders_prior.shape)
orders_prior.head()

Merged (orders + prior) shape: (32434489, 10)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


Now we are going to merge this new table with products table by "product_id"

In [8]:
orders_products = orders_prior.merge(products, on="product_id", how="left")

print("Merged (orders + products) shape:", orders_products.shape)
orders_products.head()

Merged (orders + products) shape: (32434489, 13)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17


And now we have orders with their historical register combine with order products. For the last merging we will add deparments to every product by "department_id"

In [9]:
orders_products = orders_products.merge(departments, on="department_id", how="left")

print("Final merged dataset shape:", orders_products.shape)
orders_products.head()

Final merged dataset shape: (32434489, 14)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household


now just a small fix renaming department to department_name:

In [12]:
orders_products.rename(columns={"department": "department_name"}, inplace=True)
orders_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department_name
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household


now lets have a look on unique ids number in order to makesure everything merge properly:

In [13]:
print("Number of unique users:", orders_products['user_id'].nunique())
print("Number of unique products:", orders_products['product_id'].nunique())
print("Number of unique orders:", orders_products['order_id'].nunique())
print("Number of departments:", orders_products['department_name'].nunique())

orders_products.isnull().sum()


Number of unique users: 206209
Number of unique products: 49677
Number of unique orders: 3214874
Number of departments: 21


order_id                        0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                    0
aisle_id                        0
department_id                   0
department_name                 0
dtype: int64

If we compare this unique numbers with the shape of every table we can conclude we keep the same number of products 49677, also we have the same number of orders and departments there is no information lost in the merging process, we have some missing values on days_since_prior_order but is not affecting us at the moment since this represents users first order which has no previous order.

Data Cleaning and Preprocessing
---
Now we are going to clean and prepare the data before the EDA and the ML models

In [16]:
orders_products.duplicated().sum()

np.int64(0)

No duplicated values now we aregoing to use feature engineering to create a column named "order_size" including number of items in each order:

In [17]:
# Compute number of items per order
order_size = orders_products.groupby("order_id")["product_id"].count().reset_index()
order_size.rename(columns={"product_id": "order_size"}, inplace=True)

# Merge back into the dataset
orders_products = orders_products.merge(order_size, on="order_id", how="left")

orders_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department_name,order_size
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages,5
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs,5
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks,5
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks,5
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household,5


And then we are going to create the total of orders a user had place creating the variable user_total_orders

In [18]:
user_orders = orders_products.groupby("user_id")["order_id"].nunique().reset_index()
user_orders.rename(columns={"order_id": "user_total_orders"}, inplace=True)

orders_products = orders_products.merge(user_orders, on="user_id", how="left")

orders_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department_name,order_size,user_total_orders
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages,5,10
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs,5,10
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks,5,10
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks,5,10
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household,5,10


And for last we are going to create a varible to know how popular is a product, this will give us a good insight and also will help us later on for collaborative filtering. Checking how many orders had that specific product id.

In [19]:
product_popularity = orders_products.groupby("product_id")["order_id"].count().reset_index()
product_popularity.rename(columns={"order_id": "product_popularity"}, inplace=True)

orders_products = orders_products.merge(product_popularity, on="product_id", how="left")

orders_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department_name,order_size,user_total_orders,product_popularity
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages,5,10,35791
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs,5,10,15935
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks,5,10,6476
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks,5,10,2523
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household,5,10,1214


Next we wil have a conversion of data types, but first we will check with columns needs a convertion:

In [21]:
orders_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
 7   product_id              int64  
 8   add_to_cart_order       int64  
 9   reordered               int64  
 10  product_name            object 
 11  aisle_id                int64  
 12  department_id           int64  
 13  department_name         object 
 14  order_size              int64  
 15  user_total_orders       int64  
 16  product_popularity      int64  
dtypes: float64(1), int64(13), object(3)
memory usage: 4.1+ GB


In [23]:
orders_products["department_name"] = orders_products["department_name"].astype("category")
orders_products["product_name"] = orders_products["product_name"].astype("category")
orders_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 17 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   eval_set                object  
 3   order_number            int64   
 4   order_dow               int64   
 5   order_hour_of_day       int64   
 6   days_since_prior_order  float64 
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  product_name            category
 11  aisle_id                int64   
 12  department_id           int64   
 13  department_name         category
 14  order_size              int64   
 15  user_total_orders       int64   
 16  product_popularity      int64   
dtypes: category(2), float64(1), int64(13), object(1)
memory usage: 3.8+ GB


Sampling
---
Since we do have to many information in this dataset to prevent notebook crashes we will get a sample of 5K users instead of the 20K

In [24]:
# Number of users to sample
N_USERS = 5000

# Randomly sample users
sample_users = orders_products['user_id'].drop_duplicates().sample(N_USERS, random_state=42)

# Filter dataset to only these users
orders_products_small = orders_products[orders_products['user_id'].isin(sample_users)].copy()

print("Reduced dataset shape:", orders_products_small.shape)
print("Unique users:", orders_products_small['user_id'].nunique())
print("Unique orders:", orders_products_small['order_id'].nunique())
print("Unique products:", orders_products_small['product_id'].nunique())

Reduced dataset shape: (778814, 17)
Unique users: 5000
Unique orders: 77862
Unique products: 28966
